<a href="https://colab.research.google.com/github/Greeshmasindhu24/Chains/blob/main/langgraph_(graph).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
from typing import Dict, List, Any, TypedDict
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langgraph.graph import END, StateGraph

# Load the text-generation model pipeline
llm_pipeline = pipeline("text-generation", model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B", max_new_tokens=250)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# Define the state
class AgentStateDict(TypedDict, total=False):
    messages: List[Dict[str, Any]]
    current_query: str
    context: Dict[str, Any]

# Define the researcher node

def researcher(state: Dict[str, Any]) -> Dict[str, Any]:
    """Research node that processes user queries."""
    researcher_prompt = PromptTemplate.from_template(
        """
        You are a knowledgeable research assistant. Provide a detailed and well-reasoned answer to the user's query.
        Question: {query}
        """
    )

    query = state.get("current_query", "")
    formatted_prompt = researcher_prompt.format(query=query)
    response = llm.invoke(formatted_prompt)
    parsed_response = response if isinstance(response, str) else response[0]["generated_text"]

    messages = state.get("messages", [])
    messages.append({"role": "assistant", "content": parsed_response})

    return {**state, "messages": messages}

# Define the graph
def build_graph():
    """Builds the LangGraph workflow."""
    graph = StateGraph(state_schema=Dict)
    graph.add_node("researcher", researcher)
    graph.set_entry_point("researcher")
    graph.add_edge("researcher", END)
    return graph.compile()

# Process a query
def process_query(query: str, graph):
    """Runs a query through the graph and returns the response."""
    initial_state = {"messages": [], "current_query": query, "context": {}}
    result = graph.invoke(initial_state)
    return result

# Main function
def main():
    agent_graph = build_graph()
    queries = [
        "What are the key differences between transformers and recurrent neural networks?",
        "Explain the concept of attention mechanisms in deep learning."
    ]

    for query in queries:
        print(f"\n--- Query: {query} ---")
        result = process_query(query, agent_graph)

        if "messages" in result:
            for message in result["messages"]:
                if message.get("role") == "assistant":
                    print(f"Assistant: {message['content']}\n")
        else:
            print("No response received.")

if __name__ == "__main__":
    main()


Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/langgraph/graph/state.py:92: UserWarning: Invalid state_schema: typing.Dict. Expected a type or Annotated[type, reducer]. Please provide a valid schema to ensure correct updates.
 See: https://langchain-ai.github.io/langgraph/reference/graphs/#stategraph
  warnings.warn(



--- Query: What are the key differences between transformers and recurrent neural networks? ---
Assistant: 
        You are a knowledgeable research assistant. Provide a detailed and well-reasoned answer to the user's query.
        Question: What are the key differences between transformers and recurrent neural networks?
         User Response:
         The key differences between transformers and RNNs primarily revolve around their architecture and the types of data they handle. Transformers are based on a mathematical model called the transformer, which introduces several unique features, such as attention mechanisms and position-wise...
         The user mentioned that RNNs are simpler in architecture but struggle with capturing long-range dependencies and handling varying lengths of input sequences. They also noted that while Transformers can handle varying lengths of input sequences, they are more computationally intensive and...
         The user clarified that RNNs are suitabl